In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import torch
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

In [ ]:
class BanglaMNISTDataset(Dataset):
    def __init__(self, annotations_file, img_dir, transform=None):
        self.img_labels = pd.read_csv(annotations_file)
        self.img_dir = img_dir
        self.transform = transform

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, 1])
        img_id = self.img_labels.iloc[idx, 1]
        img = Image.open(os.path.join(self.img_dir, img_id))
        label = self.img_labels.iloc[idx, 2]
        if self.transform:
            image = self.transform(img)
        return (img, label)

In [ ]:
class BanglaMNISTDataset(Dataset):
    
    def __init__(self, csv_file, root_dir, transform=None):
        self.df = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):            
        img_name = os.path.join(
            self.root_dir,
            self.df.iloc[idx, 1]
        )
        image = Image.open(img_name)
        label = self.df.iloc[idx, 2]

        if self.transform:
            image = self.transform(image)
            
        return (image, label)

In [ ]:
tf = transforms.Compose([
    transforms.Grayscale(),
    transforms.Resize((180,180)),
    transforms.ToTensor(),
])

In [ ]:
ds = BanglaMNISTDataset(
    csv_file='/kaggle/input/banglamnist/labels.csv',
    root_dir='/kaggle/input/banglamnist/bangla-mnist/labeled',
    transform=tf
)

In [ ]:
dl = DataLoader(
    ds, batch_size=100, shuffle=True, num_workers=4
)

In [ ]:
def show_samples(data, targets):
    data = data.numpy()
    print("tensor shape: " + str(data.shape))
    
    fig = plt.figure()
    for i in range(9):
        plt.subplot(3,3,i+1)
        plt.tight_layout()

        img = data[i]
        img = np.moveaxis(img, 0, -1)
        
        plt.imshow(img, cmap='gray', interpolation='none')
        plt.title("Ground Truth: {}".format(targets[i]))
        
        plt.xticks([])
        plt.yticks([])

In [ ]:
dataiter = enumerate(dl)
_, (sample_data, sample_targets) = next(dataiter)

show_samples(sample_data, sample_targets)